# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 21 2023 10:49AM,261567,10,PRF-58919,Bio-PRF,Executing
1,Apr 21 2023 10:49AM,261567,10,PRF-58923,Bio-PRF,Executing
2,Apr 21 2023 10:49AM,261567,10,PRF-58926,Bio-PRF,Executing
3,Apr 21 2023 10:49AM,261567,10,PRF-58945,Bio-PRF,Executing
4,Apr 21 2023 10:49AM,261567,10,PRF-58948,Bio-PRF,Released
5,Apr 21 2023 10:49AM,261567,10,PRF-58951,Bio-PRF,Executing
6,Apr 21 2023 10:49AM,261567,10,PRF-58993,Bio-PRF,Released
7,Apr 21 2023 10:49AM,261567,10,PRF-58996,Bio-PRF,Released
8,Apr 21 2023 10:49AM,261567,10,PRF-59000,Bio-PRF,Released
9,Apr 21 2023 10:49AM,261567,10,PRF-59006,Bio-PRF,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,261565,Executing,9
12,261565,Released,2
13,261567,Executing,8
14,261567,Released,13
15,261568,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261554,NaN,2.0
261555,NaN,1.0
261565,9.0,2.0
261567,8.0,13.0
261568,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261505,1.0,0.0
261514,0.0,2.0
261520,0.0,1.0
261534,0.0,1.0
261544,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261505,1,0
261514,0,2
261520,0,1
261534,0,1
261544,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261505,1,0
1,261514,0,2
2,261520,0,1
3,261534,0,1
4,261544,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261505,1,
1,261514,,2
2,261520,,1
3,261534,,1
4,261544,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 21 2023 10:49AM,261567,10,Bio-PRF
21,Apr 21 2023 10:47AM,261565,10,Eywa Pharma Inc.
32,Apr 21 2023 10:46AM,261568,19,"NAPP Technologies, LLC"
33,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC"
34,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC"
36,Apr 21 2023 9:03AM,261551,10,"Citieffe, Inc."
37,Apr 21 2023 9:02AM,261550,10,Emerginnova
39,Apr 21 2023 8:55AM,261549,10,"NBTY Global, Inc."
40,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc."
43,Apr 21 2023 8:21AM,261544,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 21 2023 10:49AM,261567,10,Bio-PRF,8,13
1,Apr 21 2023 10:47AM,261565,10,Eywa Pharma Inc.,9,2
2,Apr 21 2023 10:46AM,261568,19,"NAPP Technologies, LLC",,1
3,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",,1
4,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",,2
5,Apr 21 2023 9:03AM,261551,10,"Citieffe, Inc.",,1
6,Apr 21 2023 9:02AM,261550,10,Emerginnova,,2
7,Apr 21 2023 8:55AM,261549,10,"NBTY Global, Inc.",1,
8,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",,3
9,Apr 21 2023 8:21AM,261544,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 10:49AM,261567,10,Bio-PRF,13,8
1,Apr 21 2023 10:47AM,261565,10,Eywa Pharma Inc.,2,9
2,Apr 21 2023 10:46AM,261568,19,"NAPP Technologies, LLC",1,
3,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",1,
4,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",2,
5,Apr 21 2023 9:03AM,261551,10,"Citieffe, Inc.",1,
6,Apr 21 2023 9:02AM,261550,10,Emerginnova,2,
7,Apr 21 2023 8:55AM,261549,10,"NBTY Global, Inc.",,1
8,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",3,
9,Apr 21 2023 8:21AM,261544,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 10:49AM,261567,10,Bio-PRF,13,8
1,Apr 21 2023 10:47AM,261565,10,Eywa Pharma Inc.,2,9
2,Apr 21 2023 10:46AM,261568,19,"NAPP Technologies, LLC",1,
3,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",1,
4,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 10:49AM,261567,10,Bio-PRF,13.0,8.0
1,Apr 21 2023 10:47AM,261565,10,Eywa Pharma Inc.,2.0,9.0
2,Apr 21 2023 10:46AM,261568,19,"NAPP Technologies, LLC",1.0,NaN
3,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",1.0,NaN
4,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 10:49AM,261567,10,Bio-PRF,13.0,8.0
1,Apr 21 2023 10:47AM,261565,10,Eywa Pharma Inc.,2.0,9.0
2,Apr 21 2023 10:46AM,261568,19,"NAPP Technologies, LLC",1.0,0.0
3,Apr 21 2023 9:10AM,261555,19,"Emersa Waterbox, LLC",1.0,0.0
4,Apr 21 2023 9:08AM,261554,19,"Emersa Waterbox, LLC",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1569302,19.0,18.0
15,261514,2.0,0.0
19,1046211,5.0,0.0
20,261546,3.0,0.0
21,261505,0.0,1.0
22,261544,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1569302,19.0,18.0
1,15,261514,2.0,0.0
2,19,1046211,5.0,0.0
3,20,261546,3.0,0.0
4,21,261505,0.0,1.0
5,22,261544,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,19.0,18.0
1,15,2.0,0.0
2,19,5.0,0.0
3,20,3.0,0.0
4,21,0.0,1.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,15,Released,2.0
2,19,Released,5.0
3,20,Released,3.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Executing,18.0,0.0,0.0,0.0,1.0,0.0
Released,19.0,2.0,5.0,3.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Executing,18.0,0.0,0.0,0.0,1.0,0.0
1,Released,19.0,2.0,5.0,3.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Executing,18.0,0.0,0.0,0.0,1.0,0.0
1,Released,19.0,2.0,5.0,3.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()